In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

 To do this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn regressors. The first step is to create a function that will build and compile a Keras model, given a set of hyperparameters:

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer, metrics=["accuracy"])
    
    return model

Next, let’s create a KerasRegressor based on this build_model() function:

In [11]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

The KerasRegressor object is a thin wrapper around the Keras model built using build_model(). Since we did not specify any hyperparameters when creating it, it will use the default hyperparameters we defined in build_model(). Now we can use this object like a regular Scikit-Learn regressor: we can train it using its fit() method, then evaluate it using its score() method, and use it to make predictions using its predict() method, as you can see in the following code:

In [12]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 55us/sample - loss: 1.8240 - accuracy: 0.0028 - val_loss: 0.7398 - val_accuracy: 0.0036
Epoch 2/100
11610/11610 [==============================] - 0s 30us/sample - loss: 0.6763 - accuracy: 0.0031 - val_loss: 0.6588 - val_accuracy: 0.0036
Epoch 3/100
11610/11610 [==============================] - 0s 31us/sample - loss: 0.5697 - accuracy: 0.0031 - val_loss: 0.5856 - val_accuracy: 0.0036
Epoch 4/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5275 - accuracy: 0.0031 - val_loss: 0.5502 - val_accuracy: 0.0036
Epoch 5/100
11610/11610 [==============================] - 0s 28us/sample - loss: 0.4985 - accuracy: 0.0031 - val_loss: 0.5265 - val_accuracy: 0.0036
Epoch 6/100
11610/11610 [==============================] - 0s 28us/sample - loss: 0.4768 - accuracy: 0.0031 - val_loss: 0.5029 - val_accuracy: 0.0036
Epoch 7/100
11610/11610 [==========================

Epoch 55/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3518 - accuracy: 0.0031 - val_loss: 0.3742 - val_accuracy: 0.0036
Epoch 56/100
11610/11610 [==============================] - 0s 27us/sample - loss: 0.3476 - accuracy: 0.0031 - val_loss: 0.3812 - val_accuracy: 0.0036
Epoch 57/100
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3517 - accuracy: 0.0031 - val_loss: 0.3720 - val_accuracy: 0.0036
Epoch 58/100
11610/11610 [==============================] - 0s 27us/sample - loss: 0.3476 - accuracy: 0.0030 - val_loss: 0.3715 - val_accuracy: 0.0036
Epoch 59/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3441 - accuracy: 0.0031 - val_loss: 0.3723 - val_accuracy: 0.0036
Epoch 60/100
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3439 - accuracy: 0.0031 - val_loss: 0.3703 - val_accuracy: 0.0036
Epoch 61/100
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3430 - ac

In [13]:
mse_test = keras_reg.score(X_test, y_test)

5160/5160 [==============================] - 0s 16us/sample - loss: 0.3395 - accuracy: 0.0023


### hyperparameter optimization

Since there are many hyperparameters, it is preferable to use a randomized search rather than grid search. Let’s try to explore the number of hidden layers, the number of neurons, and the learning rate:

In [14]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.004453296577637131, n_hidden=3, n_neurons=70 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


7740/7740 [==============================] - 1s 66us/sample - loss: 1.1180 - accuracy: 0.0025 - val_loss: 0.6602 - val_accuracy: 0.0036
Epoch 2/100
7740/7740 [==============================] - 0s 37us/sample - loss: 0.5733 - accuracy: 0.0025 - val_loss: 0.5525 - val_accuracy: 0.0036
Epoch 3/100
7740/7740 [==============================] - 0s 38us/sample - loss: 0.4950 - accuracy: 0.0025 - val_loss: 0.4904 - val_accuracy: 0.0036
Epoch 4/100
7740/7740 [==============================] - 0s 37us/sample - loss: 0.4547 - accuracy: 0.0025 - val_loss: 0.5688 - val_accuracy: 0.0036
Epoch 5/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.4839 - accuracy: 0.0025 - val_loss: 0.4328 - val_accuracy: 0.0036
Epoch 6/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.4262 - accuracy: 0.0025 - val_loss: 0.4185 - val_accuracy: 0.0036
Epoch 7/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.3934 - accuracy: 0.0025 - val_loss: 0.4067

7740/7740 [==============================] - 0s 36us/sample - loss: 0.2845 - accuracy: 0.0025 - val_loss: 0.3171 - val_accuracy: 0.0036
Epoch 57/100
7740/7740 [==============================] - 0s 37us/sample - loss: 0.2830 - accuracy: 0.0025 - val_loss: 0.3188 - val_accuracy: 0.0036
Epoch 58/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.2821 - accuracy: 0.0025 - val_loss: 0.3176 - val_accuracy: 0.0036
Epoch 59/100
7740/7740 [==============================] - 0s 38us/sample - loss: 0.2808 - accuracy: 0.0025 - val_loss: 0.3198 - val_accuracy: 0.0036
Epoch 60/100
7740/7740 [==============================] - 0s 38us/sample - loss: 0.2802 - accuracy: 0.0025 - val_loss: 0.3157 - val_accuracy: 0.0036
Epoch 61/100
7740/7740 [==============================] - 0s 37us/sample - loss: 0.2778 - accuracy: 0.0025 - val_loss: 0.3118 - val_accuracy: 0.0036
Epoch 62/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.2793 - accuracy: 0.0025 - val_loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.3s remaining:    0.0s


...

7740/7740 [==============================] - 0s 37us/sample - loss: 0.3599 - accuracy: 0.0035 - val_loss: 0.3920 - val_accuracy: 0.0036
Epoch 86/100
7740/7740 [==============================] - 0s 36us/sample - loss: 0.3592 - accuracy: 0.0035 - val_loss: 0.3919 - val_accuracy: 0.0036
Epoch 87/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3583 - accuracy: 0.0035 - val_loss: 0.3913 - val_accuracy: 0.0036
Epoch 88/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3576 - accuracy: 0.0035 - val_loss: 0.3899 - val_accuracy: 0.0036
Epoch 89/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3566 - accuracy: 0.0035 - val_loss: 0.3893 - val_accuracy: 0.0036
Epoch 90/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3559 - accuracy: 0.0035 - val_loss: 0.3882 - val_accuracy: 0.0036
Epoch 91/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3552 - accuracy: 0.0035 - val_loss: 

7740/7740 [==============================] - 0s 34us/sample - loss: 0.4088 - accuracy: 0.0034 - val_loss: 0.4558 - val_accuracy: 0.0036
Epoch 39/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.4069 - accuracy: 0.0034 - val_loss: 0.4537 - val_accuracy: 0.0036
Epoch 40/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.4051 - accuracy: 0.0034 - val_loss: 0.4516 - val_accuracy: 0.0036
Epoch 41/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.4031 - accuracy: 0.0034 - val_loss: 0.4490 - val_accuracy: 0.0036
Epoch 42/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.4016 - accuracy: 0.0034 - val_loss: 0.4475 - val_accuracy: 0.0036
Epoch 43/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.4000 - accuracy: 0.0034 - val_loss: 0.4453 - val_accuracy: 0.0036
Epoch 44/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3984 - accuracy: 0.0034 - val_loss: 

7740/7740 [==============================] - 0s 35us/sample - loss: 0.3502 - accuracy: 0.0034 - val_loss: 0.3906 - val_accuracy: 0.0036
Epoch 94/100
7740/7740 [==============================] - 0s 34us/sample - loss: 0.3496 - accuracy: 0.0034 - val_loss: 0.3898 - val_accuracy: 0.0036
Epoch 95/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3489 - accuracy: 0.0034 - val_loss: 0.3890 - val_accuracy: 0.0036
Epoch 96/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3482 - accuracy: 0.0034 - val_loss: 0.3889 - val_accuracy: 0.0036
Epoch 97/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3476 - accuracy: 0.0034 - val_loss: 0.3884 - val_accuracy: 0.0036
Epoch 98/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3471 - accuracy: 0.0034 - val_loss: 0.3872 - val_accuracy: 0.0036
Epoch 99/100
7740/7740 [==============================] - 0s 35us/sample - loss: 0.3464 - accuracy: 0.0034 - val_loss: 

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.0min finished


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fd640b00950>, as the constructor either does not set or modifies parameter learning_rate

In [15]:
rnd_search_cv.best_params_

{'learning_rate': 0.027414237929535134, 'n_hidden': 2, 'n_neurons': 14}

In [16]:
rnd_search_cv.best_score_

-0.32415929865236426

In [20]:
rnd_search_cv.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [22]:
rnd_search_cv.score(X_test, y_test)

AttributeError: 'RandomizedSearchCV' object has no attribute 'scorer_'

In [12]:
model = rnd_search_cv.best_estimator_.model
model

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [ ]:
model.evaluate(X_test, y_test)